In [1]:
import pandas as pd

In [2]:
# --- Paso 1: Cargar y limpiar el mapeo curricular con forward-fill ---
mapeo = pd.read_excel(
    "Anexo - Descriptores currículo.xlsx",
    sheet_name="Esquema elementos currículo",
    usecols=[
        "Competencias específicas",
        "Descriptores del perfil de salida",
        "Criterios de evaluación",
        "Saberes básicos"
    ],
    dtype=str
)

# Forward-fill para propagar valores de celdas unidas
mapeo[
    [
        "Competencias específicas",
        "Descriptores del perfil de salida",
        "Criterios de evaluación",
    ]
] = mapeo[
    [
        "Competencias específicas",
        "Descriptores del perfil de salida",
        "Criterios de evaluación",
    ]
].ffill()

# Eliminar filas sin saber básico
mapeo = mapeo.dropna(subset=["Saberes básicos"])

In [3]:
# --- Paso 2: Agrupar para capturar múltiples asociaciones ---
agrupado = (
    mapeo.groupby("Saberes básicos")
    .agg(
        {
            "Competencias específicas": lambda vals: sorted(
                {str(v) for v in vals if pd.notna(v)}
            ),
            "Descriptores del perfil de salida": lambda vals: sorted(
                {v for v in vals if pd.notna(v)}
            ),
            "Criterios de evaluación": lambda vals: sorted(
                {v for v in vals if pd.notna(v)}
            ),
        }
    )
    .reset_index()
)

In [4]:
# Convertir a diccionarios de listas
dict_competencias = agrupado.set_index("Saberes básicos")[
    "Competencias específicas"
].to_dict()
dict_descriptores = agrupado.set_index("Saberes básicos")[
    "Descriptores del perfil de salida"
].to_dict()
dict_criterios = agrupado.set_index("Saberes básicos")[
    "Criterios de evaluación"
].to_dict()

In [ ]:
# --- Paso 3: Función para rellenar campos considerando múltiples asociaciones ---

def rellenar_campos(saberes_str):
    if pd.isna(saberes_str):
        return "", "", ""
    saberes = [s.strip() for s in saberes_str.split(", ")]
    competencias = []
    descriptores = []
    criterios = []
    for s in saberes:
        competencias.extend(dict_competencias.get(s, []))
        descriptores.extend(dict_descriptores.get(s, []))
        criterios.extend(dict_criterios.get(s, []))
    # Deduplicar y ordenar
    competencias = sorted(set(", ".join(competencias).split(", ")))
    descriptores = sorted(set(", ".join(descriptores).split(", ")))
    criterios = sorted(set(", ".join(criterios).split(", ")))
    # Devolver como cadenas
    return (
        ", ".join(competencias),
        ", ".join(descriptores),
        ", ".join(criterios),
    )

In [ ]:
# --- Paso 4: Leer archivo de entrada ---
df_input = pd.read_excel(
    "justificación currículo cartas enemigo.xlsx", index_col="Nº carta", dtype=str
)
df_input = df_input.dropna(subset=["Saberes básicos"])

In [ ]:
# --- Paso 5: Aplicar función y generar columnas nuevas ---
df_input[['Competencias específicas', 'Descriptores del perfil de salida', 'Criterios de evaluación']] = \
    df_input['Saberes básicos'].apply(lambda x: pd.Series(rellenar_campos(x)))

In [8]:
# --- Mostrar resultado al usuario ---
# tools.display_dataframe_to_user("Resultado de Mapeo Curricular Completo", df_input)

# --- Guardar a nuevo Excel ---
output_file = "cartas_output_completo.xlsx"
df_input.to_excel(output_file, index=True)